In [1]:
!pip install sentencepiece
!pip install datasets evaluate transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.1 MB/s eta 0:00:00
  

In [3]:
import pandas as pd


data = pd.read_csv("https://drive.google.com/uc?export=download&id=1jT9GZuC2CxjccaFXxh1neGAg9GGnfR5a")

In [4]:
from sklearn.model_selection import train_test_split


RANDOM_STATE = 420
TEST_SIZE = 0.2

train_df, val_df = train_test_split(data, test_size=TEST_SIZE, random_state=RANDOM_STATE)

In [5]:
print(f'Data: {len(data)}, Train: {len(train_df)}, Val: {len(val_df)}')

Data: 49098, Train: 39278, Val: 9820


In [6]:
for col in ["hate", "privacy", "sexual", "impersonation", "illegal", "advertisement", "ai", "neutral"]:
  print(f'{col}: {data[col].sum()}')

hate: 19581
privacy: 257
sexual: 468
impersonation: 257
illegal: 274
advertisement: 473
ai: 6043
neutral: 21753


In [7]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig, AdamW, get_linear_schedule_with_warmup

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class MultilabelDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]["body"]
        labels = self.data.iloc[idx][["hate", "privacy", "sexual", "impersonation", "illegal", "advertisement", "ai", "neutral"]].values.astype(float)

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            return_token_type_ids=True,
            truncation=True
        )

        return {
            "input_ids": torch.tensor(inputs["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(inputs["attention_mask"], dtype=torch.long),
            "labels": torch.tensor(labels, dtype=torch.float)
        }

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_dataset = MultilabelDataset(train_df, tokenizer)
val_dataset = MultilabelDataset(val_df, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

config = BertConfig.from_pretrained("bert-base-uncased", num_labels=8)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", config=config).to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * 50)
criterion = nn.BCEWithLogitsLoss()

patience, early_stop_counter = 2, 0
min_val_loss = float("inf")

for epoch in range(50):
    model.train()
    train_loss = 0

    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels)
            preds = torch.sigmoid(logits).round()
            val_loss += loss.item()
            val_correct += (preds == labels).sum().item()
            val_total += labels.numel()

    val_loss /= len(val_loader)
    val_acc = val_correct / val_total

    print(f"Epoch {epoch + 1}/{50}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    if val_loss < min_val_loss:
        min_val_loss = val_loss
        early_stop_counter = 0
        torch.save(model.state_dict(), "bert.pt")
    else:
        early_stop_counter += 1

    if early_stop_counter >= patience:
        print("Early stopping")
        break

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 1/50, Train Loss: 0.0884, Val Loss: 0.0551, Val Acc: 0.9817
Epoch 2/50, Train Loss: 0.0470, Val Loss: 0.0498, Val Acc: 0.9831
Epoch 3/50, Train Loss: 0.0342, Val Loss: 0.0514, Val Acc: 0.9841
Epoch 4/50, Train Loss: 0.0261, Val Loss: 0.0545, Val Acc: 0.9831
Early stopping


In [10]:
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, f1_score
import numpy as np


model.load_state_dict(torch.load("bert.pt"))

model.eval()
test_loss, test_correct, test_total = 0, 0, 0
all_preds, all_labels = [], []

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = criterion(logits, labels)
        preds = torch.sigmoid(logits).round()
        
        test_loss += loss.item()
        test_correct += (preds == labels).sum().item()
        test_total += labels.numel()

        all_preds.extend(preds.cpu().numpy().tolist())
        all_labels.extend(labels.cpu().numpy().tolist())

test_loss /= len(val_loader)
test_acc = test_correct / test_total

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

print("Classification Report for BERT:\n", classification_report(all_labels, all_preds, target_names=["hate", "privacy", "sexual", "impersonation", "illegal", "advertisement", "ai", "neutral"], zero_division=0))

acc = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {acc:.2f}')

f1 = f1_score(all_labels, all_preds, average="weighted")
print(f'F1 score: {f1:.2f}')

Classification Report for BERT:
                precision    recall  f1-score   support

         hate       0.95      0.92      0.94      3895
      privacy       0.79      0.61      0.69        49
       sexual       0.76      0.19      0.30        84
impersonation       0.96      0.49      0.65        51
      illegal       0.41      0.59      0.48        39
advertisement       1.00      0.45      0.62        95
           ai       0.97      0.97      0.97      1217
      neutral       0.93      0.94      0.94      4394

    micro avg       0.94      0.92      0.93      9824
    macro avg       0.85      0.65      0.70      9824
 weighted avg       0.94      0.92      0.93      9824
  samples avg       0.92      0.92      0.92      9824

Accuracy: 0.92
F1 score: 0.93
